<a href="https://colab.research.google.com/github/sdeepa-24/AI-lab-code/blob/main/lan_networking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
sentence = input("Enter a sentence: ")


words = sentence.split()
print("Word Tokens:", words)


characters = list(sentence)
print("Character Tokens:", characters)

Enter a sentence: Alliance University
Word Tokens: ['Alliance', 'University']
Character Tokens: ['A', 'l', 'l', 'i', 'a', 'n', 'c', 'e', ' ', 'U', 'n', 'i', 'v', 'e', 'r', 's', 'i', 't', 'y']


In [3]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

paragraph = input("Enter a paragraph: ")

sentences = nltk.sent_tokenize(paragraph)
print("Sentence Tokens:")
for s in sentences:
    print(s)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Enter a paragraph: Enter a paragraph: Natural Language Processing is interesting. It is used in AI. NLP helps machines understand language.
Sentence Tokens:
Enter a paragraph: Natural Language Processing is interesting.
It is used in AI.
NLP helps machines understand language.


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

sentence = input("Enter a sentence: ")

vectorizer = TfidfVectorizer()
vector = vectorizer.fit_transform([sentence])

print("Feature Names:", vectorizer.get_feature_names_out())
print("Sentence Vector:\n", vector.toarray())


Enter a sentence: I am in Alliance University
Feature Names: ['alliance' 'am' 'in' 'university']
Sentence Vector:
 [[0.5 0.5 0.5 0.5]]


In [9]:
!pip install transformers torch scikit-learn

import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained Sentence Transformer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Example sentences
text1 = "I love machine learning"
text2 = "I enjoy studying artificial intelligence"

# Generate embeddings
emb1 = get_embedding(text1)
emb2 = get_embedding(text2)

# Cosine similarity
similarity = cosine_similarity(emb1, emb2)

print("Text Similarity Score:", similarity[0][0])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Text Similarity Score: 0.56718445


In [10]:
!pip install transformers torch scikit-learn
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd

# Sample Dataset (replace with CSV if needed)
data = {
    "text": [
        "Government launches new education policy",
        "Aliens landed in India yesterday",
        "Stock market shows steady growth",
        "Drinking cow urine cures cancer",
        "Scientists discover new planet"
    ],
    "label": [1, 0, 1, 0, 1]  # 1 = Real, 0 = Fake
}

df = pd.DataFrame(data)

# Load tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

# Dataset class
class FakeNewsDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=128
        )
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df["text"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42
)

train_dataset = FakeNewsDataset(X_train, y_train)
test_dataset = FakeNewsDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Training loop (1 epoch – demo)
model.train()
for batch in train_loader:
    optimizer.zero_grad()
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

print("Training completed")

# Prediction
model.eval()
news = ["NASA confirms water on Mars"]
inputs = tokenizer(news, return_tensors="pt", truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1)

print("Prediction:", "Real News" if prediction.item() == 1 else "Fake News")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training completed
Prediction: Real News


In [11]:
!pip install transformers torch
from transformers import pipeline

# Load sentiment analysis pipeline
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

# Sample texts
texts = [
    "I love this product, it works perfectly!",
    "This is the worst experience ever",
    "The movie was okay, not bad",
    "I am extremely happy with the service",
    "I hate waiting in long queues"
]

# Analyze sentiment
for text in texts:
    result = sentiment_analyzer(text)[0]
    print("Text:", text)
    print("Sentiment:", result["label"])
    print("Confidence:", round(result["score"], 3))
    print()


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Text: I love this product, it works perfectly!
Sentiment: POSITIVE
Confidence: 1.0

Text: This is the worst experience ever
Sentiment: NEGATIVE
Confidence: 1.0

Text: The movie was okay, not bad
Sentiment: POSITIVE
Confidence: 0.999

Text: I am extremely happy with the service
Sentiment: POSITIVE
Confidence: 1.0

Text: I hate waiting in long queues
Sentiment: NEGATIVE
Confidence: 0.997

